In [ ]:
# 📘 Gemini 1.5 Flash: AI Art Portfolio Generator using Google Generative AI API

## ✅ Step 1: Install Google Generative AI SDK
!pip install -q google-generativeai

## ✅ Step 2: Import Libraries
import google.generativeai as genai
import PIL.Image
import io
from IPython.display import display
import os

## ✅ Step 3: Configure Gemini API Key
# Paste your API key below
api_key = "your_api_key_here"  # 🔁 Replace this with your actual Gemini API key
genai.configure(api_key=api_key)

model = genai.GenerativeModel("gemini-1.5-flash")

## ✅ Step 4: Upload Image
from google.colab import files

uploaded = files.upload()
file_name = list(uploaded.keys())[0]

image = PIL.Image.open(file_name)
display(image)

## ✅ Step 5: Create Prompt for Art Portfolio Description
prompt = """You're an expert creative content writer and SEO specialist for artists.
Given the artwork image, generate the following:
- A catchy and professional artwork title
- A 2-3 sentence art description for a portfolio or gallery
- 5 relevant SEO tags
Make it poetic and audience-friendly.
"""

## ✅ Step 6: Send Image + Prompt to Gemini Model
response = model.generate_content([prompt, image], stream=False)
print("🔍 Generated Output:")
print(response.text)

## ✅ Step 7 (Optional): Convert to Structured JSON Output
import re
import json

def extract_json(gemini_output):
    title = re.search(r"(?i)title[:\-\n]*([^\n]+)", gemini_output)
    desc = re.search(r"(?i)description[:\-\n]*([\s\S]+?)tags[:\n]", gemini_output)
    tags = re.search(r"(?i)tags[:\-\n]*([^\n]+)", gemini_output)
    return {
        "title": title.group(1).strip() if title else "",
        "description": desc.group(1).strip() if desc else "",
        "tags": [tag.strip() for tag in tags.group(1).split(",")] if tags else []
    }

result = extract_json(response.text)
print("\n📦 Structured Output:")
print(json.dumps(result, indent=2))

Saving image-from-rawpixel-id-426278-jpeg (1).jpg to image-from-rawpixel-id-426278-jpeg (1).jpg
Buffered data was truncated after reaching the output size limit.

In [ ]:
!pip install google-generativeai fastapi uvicorn jinja2 python-multipart aiofiles


In [ ]:
# ai_utils.py
import google.generativeai as genai
import os

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

model = genai.GenerativeModel("gemini-1.5-flash")

def generate_portfolio_section(prompt: str) -> str:
    response = model.generate_content(prompt)
    return response.text.strip()


In [ ]:
# main.py
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from starlette.templating import Jinja2Templates
from starlette.requests import Request
import os

import google.generativeai as genai

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

model = genai.GenerativeModel("gemini-1.5-flash")

def generate_portfolio_section(prompt: str) -> str:
    response = model.generate_content(prompt)
    return response.text.strip()


app = FastAPI()
os.makedirs("static", exist_ok=True) # Create the static directory
os.makedirs("templates", exist_ok=True) # Create the templates directory
app.mount("/static", StaticFiles(directory="static"), name="static")
templates = Jinja2Templates(directory="templates")

# Create the portfolio.html file
portfolio_html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>{{ name }} | Portfolio</title>
  <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
</head>
<body class="bg-white text-gray-800">
  <section class="p-8 max-w-5xl mx-auto">
    <h1 class="text-4xl font-bold">{{ name }}</h1>
    <p class="mt-2 italic text-sm">{{ bio }}</p>

    <div class="mt-6">
      <h2 class="text-2xl font-semibold">About Me</h2>
      <p class="mt-2">{{ about }}</p>
    </div>

    <div class="mt-6">
      <h2 class="text-2xl font-semibold">Art Style: {{ style }}</h2>
      <p class="mt-2">{{ description }}</p>
    </div>

    <div class="mt-6 grid grid-cols-2 gap-4">
      {% for img in images %}
      <img src="{{ img }}" class="rounded-xl shadow" />
      {% endfor %}
    </div>
  </section>
</body>
</html>
"""
with open("templates/portfolio.html", "w") as f:
    f.write(portfolio_html_content)


@app.post("/generate")
async def generate_site(
    request: Request,
    name: str = Form(...),
    bio: str = Form(...),
    art_style: str = Form(...),
    files: list[UploadFile] = File(...)
):
    os.makedirs("static/images", exist_ok=True)
    image_urls = []

    for file in files:
        path = f"static/images/{file.filename}"
        with open(path, "wb") as f:
            f.write(await file.read())
        image_urls.append(path)

    about_text = generate_portfolio_section(f"Write an engaging 'About the Artist' section for an artist named {name}, who specializes in {art_style}. Use an elegant tone.")
    description_text = generate_portfolio_section(f"Write a short art style summary for {art_style}. Focus on what makes this craft meaningful.")

    return templates.TemplateResponse("portfolio.html", {
        "request": request,
        "name": name,
        "bio": bio,
        "about": about_text,
        "style": art_style,
        "description": description_text,
        "images": image_urls
    })


# Create Your Art Portfolio

<form action="/generate" method="post" enctype="multipart/form-data">
  <div>
    <label for="name">Artist Name:</label>
    <input type="text" id="name" name="name" required>
  </div>
  <div>
    <label for="bio">Short Bio:</label>
    <textarea id="bio" name="bio" required></textarea>
  </div>
  <div>
    <label for="art_style">Art Style:</label>
    <input type="text" id="art_style" name="art_style" required>
  </div>
  <div>
    <label for="files">Upload Images:</label>
    <input type="file" id="files" name="files" multiple accept="image/*" required>
  </div>
  <button type="submit">Generate Portfolio</button>
</form>

# Task
Generate Python code using FastAPI and the latest Gemini Flash 1.5 API to create a portfolio website generator. The application should take user input (name, bio, art style) and multiple image uploads, use AI to generate content (titles, descriptions, tags) for each image based on the provided information and the images themselves, and then dynamically generate an HTML portfolio website displaying the user's information and the AI-generated content alongside the uploaded images. The code should include a basic HTML template and use Tailwind CSS for styling. The application should store uploaded images locally.

## Refactor existing code

### Subtask:
Organize the existing code into a more modular structure, separating the AI utility functions from the FastAPI application logic.


**Reasoning**:
Create the `ai_utils.py` file and move the Gemini API configuration and the `generate_portfolio_section` function into it.



In [ ]:
# ai_utils.py
import google.generativeai as genai
import os
import PIL.Image # Import PIL.Image

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

model = genai.GenerativeModel("gemini-1.5-flash")

# Modify the function to accept an optional image argument
def generate_portfolio_section(prompt: str, image: PIL.Image.Image = None) -> str:
    if image:
        response = model.generate_content([prompt, image]) # Include image in the prompt if provided
    else:
        response = model.generate_content(prompt)
    return response.text.strip()

In [ ]:
# ai_utils.py
import google.generativeai as genai
import os

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

model = genai.GenerativeModel("gemini-1.5-flash")

def generate_portfolio_section(prompt: str) -> str:
    response = model.generate_content(prompt)
    return response.text.strip()

**Reasoning**:
Now that `ai_utils.py` is created, re-run the code for `main.py` which imports from `ai_utils`.



In [ ]:
%%writefile ai_utils.py
import google.generativeai as genai
import os

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

model = genai.GenerativeModel("gemini-1.5-flash")

def generate_portfolio_section(prompt: str) -> str:
    response = model.generate_content(prompt)
    return response.text.strip()


Writing ai_utils.py


**Reasoning**:
The user provided Python code.



In [ ]:
# main.py
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from starlette.templating import Jinja2Templates
from starlette.requests import Request
import os
from ai_utils import generate_portfolio_section


app = FastAPI()
os.makedirs("static", exist_ok=True) # Create the static directory
os.makedirs("templates", exist_ok=True) # Create the templates directory
app.mount("/static", StaticFiles(directory="static"), name="static")
templates = Jinja2Templates(directory="templates")

# Create the portfolio.html file
portfolio_html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>{{ name }} | Portfolio</title>
  <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
</head>
<body class="bg-white text-gray-800">
  <section class="p-8 max-w-5xl mx-auto">
    <h1 class="text-4xl font-bold">{{ name }}</h1>
    <p class="mt-2 italic text-sm">{{ bio }}</p>

    <div class="mt-6">
      <h2 class="text-2xl font-semibold">About Me</h2>
      <p class="mt-2">{{ about }}</p>
    </div>

    <div class="mt-6">
      <h2 class="text-2xl font-semibold">Art Style: {{ style }}</h2>
      <p class="mt-2">{{ description }}</p>
    </div>

    <div class="mt-6 grid grid-cols-2 gap-4">
      {% for img in images %}
      <img src="{{ img }}" class="rounded-xl shadow" />
      {% endfor %}
    </div>
  </section>
</body>
</html>
"""
with open("templates/portfolio.html", "w") as f:
    f.write(portfolio_html_content)


@app.post("/generate")
async def generate_site(
    request: Request,
    name: str = Form(...),
    bio: str = Form(...),
    art_style: str = Form(...),
    files: list[UploadFile] = File(...)
):
    os.makedirs("static/images", exist_ok=True)
    image_urls = []

    for file in files:
        path = f"static/images/{file.filename}"
        with open(path, "wb") as f:
            f.write(await file.read())
        image_urls.append(path)

    about_text = generate_portfolio_section(f"Write an engaging 'About the Artist' section for an artist named {name}, who specializes in {art_style}. Use an elegant tone.")
    description_text = generate_portfolio_section(f"Write a short art style summary for {art_style}. Focus on what makes this craft meaningful.")

    return templates.TemplateResponse("portfolio.html", {
        "request": request,
        "name": name,
        "bio": bio,
        "about": about_text,
        "style": art_style,
        "description": description_text,
        "images": image_urls
    })


## Enhance image handling

### Subtask:
Modify the FastAPI application to correctly handle multiple image uploads and store them in a designated directory.


**Reasoning**:
Modify the `/generate` endpoint in `main.py` to correctly handle multiple image uploads, iterate through them, read their content, save them to the specified directory, and collect their paths.



In [ ]:
# main.py
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from starlette.templating import Jinja2Templates
from starlette.requests import Request
import os
from ai_utils import generate_portfolio_section
import aiofiles # Import aiofiles

app = FastAPI()
os.makedirs("static", exist_ok=True) # Create the static directory
os.makedirs("templates", exist_ok=True) # Create the templates directory
app.mount("/static", StaticFiles(directory="static"), name="static")
templates = Jinja2Templates(directory="templates")

# Create the portfolio.html file
portfolio_html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>{{ name }} | Portfolio</title>
  <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
</head>
<body class="bg-white text-gray-800">
  <section class="p-8 max-w-5xl mx-auto">
    <h1 class="text-4xl font-bold">{{ name }}</h1>
    <p class="mt-2 italic text-sm">{{ bio }}</p>

    <div class="mt-6">
      <h2 class="text-2xl font-semibold">About Me</h2>
      <p class="mt-2">{{ about }}</p>
    </div>

    <div class="mt-6">
      <h2 class="text-2xl font-semibold">Art Style: {{ style }}</h2>
      <p class="mt-2">{{ description }}</p>
    </div>

    <div class="mt-6 grid grid-cols-2 gap-4">
      {% for img in images %}
      <img src="{{ img }}" class="rounded-xl shadow" />
      {% endfor %}
    </div>
  </section>
</body>
</html>
"""
with open("templates/portfolio.html", "w") as f:
    f.write(portfolio_html_content)


@app.post("/generate")
async def generate_site(
    request: Request,
    name: str = Form(...),
    bio: str = Form(...),
    art_style: str = Form(...),
    files: list[UploadFile] = File(...)
):
    os.makedirs("static/images", exist_ok=True)
    image_urls = []

    for file in files:
        file_path = f"static/images/{file.filename}" # Construct file path
        async with aiofiles.open(file_path, 'wb') as out_file: # Use aiofiles for async writing
            content = await file.read()
            await out_file.write(content)
        image_urls.append(file_path)

    about_text = generate_portfolio_section(f"Write an engaging 'About the Artist' section for an artist named {name}, who specializes in {art_style}. Use an elegant tone.")
    description_text = generate_portfolio_section(f"Write a short art style summary for {art_style}. Focus on what makes this craft meaningful.")

    return templates.TemplateResponse("portfolio.html", {
        "request": request,
        "name": name,
        "bio": bio,
        "about": about_text,
        "style": art_style,
        "description": description_text,
        "images": image_urls
    })

## Integrate ai for image analysis

### Subtask:
Update the AI content generation to take into account the uploaded images when generating descriptions, titles, and tags for individual artworks within the portfolio.


**Reasoning**:
Update the ai_utils.py file to modify the generate_portfolio_section function to accept an optional image argument and include the image in the prompt if provided. Also, import PIL.Image.



In [ ]:
%%writefile ai_utils.py
import google.generativeai as genai
import os
import PIL.Image

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

model = genai.GenerativeModel("gemini-1.5-flash")

def generate_portfolio_section(prompt: str, image: PIL.Image.Image = None) -> str:
    if image:
        response = model.generate_content([prompt, image])
    else:
        response = model.generate_content(prompt)
    return response.text.strip()


Overwriting ai_utils.py


**Reasoning**:
Modify the /generate endpoint in main.py to iterate through the uploaded images, open each image, and call the updated generate_portfolio_section function to generate content for each image. Store the generated content with the image URL.



In [ ]:
# main.py
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from starlette.templating import Jinja2Templates
from starlette.requests import Request
import os
from ai_utils import generate_portfolio_section
import aiofiles
import PIL.Image
import io


app = FastAPI()
os.makedirs("static", exist_ok=True)
os.makedirs("templates", exist_ok=True)
app.mount("/static", StaticFiles(directory="static"), name="static")
templates = Jinja2Templates(directory="templates")

# Create the portfolio.html file
portfolio_html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>{{ name }} | Portfolio</title>
  <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
</head>
<body class="bg-white text-gray-800">
  <section class="p-8 max-w-5xl mx-auto">
    <h1 class="text-4xl font-bold">{{ name }}</h1>
    <p class="mt-2 italic text-sm">{{ bio }}</p>

    <div class="mt-6">
      <h2 class="text-2xl font-semibold">About Me</h2>
      <p class="mt-2">{{ about }}</p>
    </div>

    <div class="mt-6">
      <h2 class="text-2xl font-semibold">Art Style: {{ style }}</h2>
      <p class="mt-2">{{ description }}</p>
    </div>

    <div class="mt-6 grid grid-cols-2 gap-4">
      {% for artwork in artworks %}
      <div>
        <img src="{{ artwork.url }}" class="rounded-xl shadow" />
        <h3 class="mt-2 text-xl font-semibold">{{ artwork.title }}</h3>
        <p class="mt-1 text-sm">{{ artwork.description }}</p>
        <p class="mt-1 text-xs text-gray-600">Tags: {{ artwork.tags | join(', ') }}</p>
      </div>
      {% endfor %}
    </div>
  </section>
</body>
</html>
"""
with open("templates/portfolio.html", "w") as f:
    f.write(portfolio_html_content)


@app.post("/generate")
async def generate_site(
    request: Request,
    name: str = Form(...),
    bio: str = Form(...),
    art_style: str = Form(...),
    files: list[UploadFile] = File(...)
):
    os.makedirs("static/images", exist_ok=True)
    artworks_data = []

    for file in files:
        file_path = f"static/images/{file.filename}"
        async with aiofiles.open(file_path, 'wb') as out_file:
            content = await file.read()
            await out_file.write(content)

        image = PIL.Image.open(io.BytesIO(content))

        artwork_prompt = f"""You're an expert creative content writer and SEO specialist for artists.
Given the artwork image, generate the following:
- A catchy and professional artwork title
- A 2-3 sentence art description for a portfolio or gallery
- 5 relevant SEO tags
Make it poetic and audience-friendly.
"""
        artwork_content = generate_portfolio_section(artwork_prompt, image=image)

        # Basic parsing of the generated content - refine this based on actual model output format
        import re
        title = "Generated Title"
        description = "Generated Description"
        tags = ["tag1", "tag2"] # Example tags

        title_match = re.search(r"(?i)title[:\-\n]*([^\n]+)", artwork_content)
        desc_match = re.search(r"(?i)description[:\-\n]*([\s\S]+?)tags[:\n]", artwork_content)
        tags_match = re.search(r"(?i)tags[:\-\n]*([^\n]+)", artwork_content)

        if title_match:
            title = title_match.group(1).strip()
        if desc_match:
            description = desc_match.group(1).strip()
        if tags_match:
            tags = [tag.strip() for tag in tags_match.group(1).split(",")]

        artworks_data.append({
            "url": file_path,
            "title": title,
            "description": description,
            "tags": tags
        })


    about_text = generate_portfolio_section(f"Write an engaging 'About the Artist' section for an artist named {name}, who specializes in {art_style}. Use an elegant tone.")
    description_text = generate_portfolio_section(f"Write a short art style summary for {art_style}. Focus on what makes this craft meaningful.")

    return templates.TemplateResponse("portfolio.html", {
        "request": request,
        "name": name,
        "bio": bio,
        "about": about_text,
        "style": art_style,
        "description": description_text,
        "artworks": artworks_data # Pass the list of artwork data
    })

## Generate dynamic portfolio content

### Subtask:
Modify the HTML template and the FastAPI endpoint to dynamically generate content for each uploaded image, including AI-generated descriptions, titles, and tags.


## Implement a user interface for input

### Subtask:
Create an HTML form for users to input their name, bio, art style, and upload images.


**Reasoning**:
Create the index.html file with the necessary form elements and modify main.py to serve this file at the root URL.



In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>Generate Your Art Portfolio</title>
  <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
</head>
<body class="bg-white text-gray-800 p-8">
  <div class="max-w-2xl mx-auto">
    <h1 class="text-3xl font-bold mb-6">Generate Your Art Portfolio</h1>

    <form action="/generate" method="post" enctype="multipart/form-data" class="space-y-4">
      <div>
        <label for="name" class="block text-sm font-medium text-gray-700">Artist Name:</label>
        <input type="text" id="name" name="name" required class="mt-1 block w-full rounded-md border-gray-300 shadow-sm focus:border-indigo-500 focus:ring-indigo-500 sm:text-sm">
      </div>

      <div>
        <label for="bio" class="block text-sm font-medium text-gray-700">Short Bio:</label>
        <textarea id="bio" name="bio" rows="3" required class="mt-1 block w-full rounded-md border-gray-300 shadow-sm focus:border-indigo-500 focus:ring-indigo-500 sm:text-sm"></textarea>
      </div>

      <div>
        <label for="art_style" class="block text-sm font-medium text-gray-700">Art Style:</label>
        <input type="text" id="art_style" name="art_style" required class="mt-1 block w-full rounded-md border-gray-300 shadow-sm focus:border-indigo-500 focus:ring-indigo-500 sm:text-sm">
      </div>

      <div>
        <label for="files" class="block text-sm font-medium text-gray-700">Upload Images:</label>
        <input type="file" id="files" name="files" multiple accept="image/*" required class="mt-1 block w-full text-sm text-gray-500 file:mr-4 file:py-2 file:px-4 file:rounded-full file:border-0 file:text-sm file:font-semibold file:bg-blue-50 file:text-blue-700 hover:file:bg-blue-100">
      </div>

      <div>
        <button type="submit" class="w-full flex justify-center py-2 px-4 border border-transparent rounded-md shadow-sm text-sm font-medium text-white bg-indigo-600 hover:bg-indigo-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-indigo-500">
          Generate Portfolio
        </button>
      </div>
    </form>
  </div>
</body>
</html>

Writing templates/index.html


**Reasoning**:
Add a FastAPI endpoint in main.py to serve the index.html template at the root URL.



In [ ]:
# main.py
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from starlette.templating import Jinja2Templates
from starlette.requests import Request
import os
from ai_utils import generate_portfolio_section
import aiofiles
import PIL.Image
import io


app = FastAPI()
os.makedirs("static", exist_ok=True)
os.makedirs("templates", exist_ok=True)
app.mount("/static", StaticFiles(directory="static"), name="static")
templates = Jinja2Templates(directory="templates")

# Create the portfolio.html file
portfolio_html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>{{ name }} | Portfolio</title>
  <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
</head>
<body class="bg-white text-gray-800">
  <section class="p-8 max-w-5xl mx-auto">
    <h1 class="text-4xl font-bold">{{ name }}</h1>
    <p class="mt-2 italic text-sm">{{ bio }}</p>

    <div class="mt-6">
      <h2 class="text-2xl font-semibold">About Me</h2>
      <p class="mt-2">{{ about }}</p>
    </div>

    <div class="mt-6">
      <h2 class="text-2xl font-semibold">Art Style: {{ style }}</h2>
      <p class="mt-2">{{ description }}</p>
    </div>

    <div class="mt-6 grid grid-cols-2 gap-4">
      {% for artwork in artworks %}
      <div>
        <img src="{{ artwork.url }}" class="rounded-xl shadow" />
        <h3 class="mt-2 text-xl font-semibold">{{ artwork.title }}</h3>
        <p class="mt-1 text-sm">{{ artwork.description }}</p>
        <p class="mt-1 text-xs text-gray-600">Tags: {{ artwork.tags | join(', ') }}</p>
      </div>
      {% endfor %}
    </div>
  </section>
</body>
</html>
"""
with open("templates/portfolio.html", "w") as f:
    f.write(portfolio_html_content)


@app.get("/", response_class=HTMLResponse)
async def read_root(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})


@app.post("/generate")
async def generate_site(
    request: Request,
    name: str = Form(...),
    bio: str = Form(...),
    art_style: str = Form(...),
    files: list[UploadFile] = File(...)
):
    os.makedirs("static/images", exist_ok=True)
    artworks_data = []

    for file in files:
        file_path = f"static/images/{file.filename}"
        async with aiofiles.open(file_path, 'wb') as out_file:
            content = await file.read()
            await out_file.write(content)

        image = PIL.Image.open(io.BytesIO(content))

        artwork_prompt = f"""You're an expert creative content writer and SEO specialist for artists.
Given the artwork image, generate the following:
- A catchy and professional artwork title
- A 2-3 sentence art description for a portfolio or gallery
- 5 relevant SEO tags
Make it poetic and audience-friendly.
"""
        artwork_content = generate_portfolio_section(artwork_prompt, image=image)

        # Basic parsing of the generated content - refine this based on actual model output format
        import re
        title = "Generated Title"
        description = "Generated Description"
        tags = ["tag1", "tag2"] # Example tags

        title_match = re.search(r"(?i)title[:\-\n]*([^\n]+)", artwork_content)
        desc_match = re.search(r"(?i)description[:\-\n]*([\s\S]+?)tags[:\n]", artwork_content)
        tags_match = re.search(r"(?i)tags[:\-\n]*([^\n]+)", artwork_content)

        if title_match:
            title = title_match.group(1).strip()
        if desc_match:
            description = desc_match.group(1).strip()
        if tags_match:
            tags = [tag.strip() for tag in tags_match.group(1).split(",")]

        artworks_data.append({
            "url": file_path,
            "title": title,
            "description": description,
            "tags": tags
        })


    about_text = generate_portfolio_section(f"Write an engaging 'About the Artist' section for an artist named {name}, who specializes in {art_style}. Use an elegant tone.")
    description_text = generate_portfolio_section(f"Write a short art style summary for {art_style}. Focus on what makes this craft meaningful.")

    return templates.TemplateResponse("portfolio.html", {
        "request": request,
        "name": name,
        "bio": bio,
        "about": about_text,
        "style": art_style,
        "description": description_text,
        "artworks": artworks_data
    })

## Connect frontend and backend

### Subtask:
Ensure the HTML form correctly sends data to the FastAPI endpoint and displays the generated portfolio.


**Reasoning**:
Review the `index.html` file and the `/generate` endpoint in `main.py` to ensure the form correctly sends data to the FastAPI endpoint and displays the generated portfolio, following steps 1-6 of the instructions.



In [ ]:
# Review of index.html and main.py based on instructions 1-6

# Step 1 & 2: Check form action, method, and enctype in index.html
# The index.html file already has:
# <form action="/generate" method="post" enctype="multipart/form-data" class="space-y-4">
# This correctly sets the action to /generate, method to post, and enctype for file uploads.

# Step 3: Confirm name attributes in index.html match parameter names in /generate endpoint
# The input fields in index.html have name attributes: name, bio, art_style, files.
# The /generate endpoint in main.py has parameters: name: str = Form(...), bio: str = Form(...), art_style: str = Form(...), files: list[UploadFile] = File(...).
# The name attributes match the parameter names.

# Step 4: Examine /generate endpoint for receiving form data and files
# The /generate endpoint uses Form(...) for name, bio, and art_style, and File(...) for files.
# This confirms it receives the form data and uploaded files correctly.

# Step 5: Ensure /generate endpoint renders portfolio.html and passes necessary data
# The /generate endpoint returns templates.TemplateResponse("portfolio.html", {...}).
# It passes name, bio, about_text (as about), art_style (as style), description_text (as description), and artworks_data (as artworks) to the template context.
# This confirms it renders portfolio.html and passes the necessary data.

# Step 6: Check portfolio.html template for accessing and displaying data
# The portfolio.html template accesses {{ name }}, {{ bio }}, {{ about }}, {{ style }}, {{ description }}.
# It iterates through artworks using {% for artwork in artworks %} and accesses artwork.url, artwork.title, artwork.description, and artwork.tags.
# This confirms it correctly accesses and displays the data passed from the /generate endpoint.

print("Review complete. The index.html form and main.py /generate endpoint appear to be correctly configured based on the instructions.")


Review complete. The index.html form and main.py /generate endpoint appear to be correctly configured based on the instructions.


## Add styling and structure

### Subtask:
Enhance the visual appeal and structure of the generated portfolio website using CSS (Tailwind CSS is already included).


**Reasoning**:
Review the existing portfolio.html template and add Tailwind CSS classes to enhance the visual appearance and structure, improving spacing, typography, and layout for better presentation of the artwork and artist information.



In [ ]:
portfolio_html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>{{ name }} | Portfolio</title>
  <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
</head>
<body class="bg-gray-100 text-gray-800 font-sans leading-normal tracking-normal">
  <header class="bg-white shadow py-6">
    <div class="container mx-auto px-4 max-w-5xl">
      <h1 class="text-4xl font-bold text-gray-900">{{ name }}</h1>
      <p class="mt-2 italic text-sm text-gray-600">{{ bio }}</p>
    </div>
  </header>

  <main class="container mx-auto px-4 py-8 max-w-5xl">
    <section class="mb-12">
      <h2 class="text-3xl font-semibold text-gray-900 mb-4 border-b-2 border-indigo-500 pb-2">About Me</h2>
      <p class="text-lg text-gray-700 leading-relaxed">{{ about }}</p>
    </section>

    <section class="mb-12">
      <h2 class="text-3xl font-semibold text-gray-900 mb-4 border-b-2 border-indigo-500 pb-2">Art Style: {{ style }}</h2>
      <p class="text-lg text-gray-700 leading-relaxed">{{ description }}</p>
    </section>

    <section>
      <h2 class="text-3xl font-semibold text-gray-900 mb-6 border-b-2 border-indigo-500 pb-2">Gallery</h2>
      <div class="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-8">
        {% for artwork in artworks %}
        <div class="bg-white rounded-xl shadow-lg overflow-hidden transform transition duration-500 hover:scale-105">
          <img src="{{ artwork.url }}" class="w-full h-64 object-cover" alt="Artwork by {{ name }}" />
          <div class="p-6">
            <h3 class="text-xl font-semibold text-gray-900 mb-2">{{ artwork.title }}</h3>
            <p class="text-gray-700 text-sm mb-4">{{ artwork.description }}</p>
            <p class="text-xs text-gray-600">Tags: <span class="font-medium">{{ artwork.tags | join(', ') }}</span></p>
          </div>
        </div>
        {% endfor %}
      </div>
    </section>
  </main>

  <footer class="bg-gray-800 text-white py-6 mt-12">
    <div class="container mx-auto px-4 text-center text-gray-400">
      <p>&copy; {{ name }} All rights reserved.</p>
    </div>
  </footer>
</body>
</html>
"""
with open("templates/portfolio.html", "w") as f:
    f.write(portfolio_html_content)

## Fine-tune content generation (optional but recommended)

### Subtask:
Explore ways to further fine-tune the content generation based on user preferences or specific artistic styles.


**Reasoning**:
Modify the `artwork_prompt` in `main.py` to include the user's `art_style` and add conditional logic for specific artistic styles to fine-tune content generation.



In [ ]:
# main.py
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from starlette.templating import Jinja2Templates
from starlette.requests import Request
import os
from ai_utils import generate_portfolio_section
import aiofiles
import PIL.Image
import io
import re


app = FastAPI()
os.makedirs("static", exist_ok=True)
os.makedirs("templates", exist_ok=True)
app.mount("/static", StaticFiles(directory="static"), name="static")
templates = Jinja2Templates(directory="templates")

# Create the portfolio.html file (using the enhanced version from the previous step)
portfolio_html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>{{ name }} | Portfolio</title>
  <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
</head>
<body class="bg-gray-100 text-gray-800 font-sans leading-normal tracking-normal">
  <header class="bg-white shadow py-6">
    <div class="container mx-auto px-4 max-w-5xl">
      <h1 class="text-4xl font-bold text-gray-900">{{ name }}</h1>
      <p class="mt-2 italic text-sm text-gray-600">{{ bio }}</p>
    </div>
  </header>

  <main class="container mx-auto px-4 py-8 max-w-5xl">
    <section class="mb-12">
      <h2 class="text-3xl font-semibold text-gray-900 mb-4 border-b-2 border-indigo-500 pb-2">About Me</h2>
      <p class="text-lg text-gray-700 leading-relaxed">{{ about }}</p>
    </section>

    <section class="mb-12">
      <h2 class="text-3xl font-semibold text-gray-900 mb-4 border-b-2 border-indigo-500 pb-2">Art Style: {{ style }}</h2>
      <p class="text-lg text-gray-700 leading-relaxed">{{ description }}</p>
    </section>

    <section>
      <h2 class="text-3xl font-semibold text-gray-900 mb-6 border-b-2 border-indigo-500 pb-2">Gallery</h2>
      <div class="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-8">
        {% for artwork in artworks %}
        <div class="bg-white rounded-xl shadow-lg overflow-hidden transform transition duration-500 hover:scale-105">
          <img src="{{ artwork.url }}" class="w-full h-64 object-cover" alt="Artwork by {{ name }}" />
          <div class="p-6">
            <h3 class="text-xl font-semibold text-gray-900 mb-2">{{ artwork.title }}</h3>
            <p class="text-gray-700 text-sm mb-4">{{ artwork.description }}</p>
            <p class="text-xs text-gray-600">Tags: <span class="font-medium">{{ artwork.tags | join(', ') }}</span></p>
          </div>
        </div>
        {% endfor %}
      </div>
    </section>
  </main>

  <footer class="bg-gray-800 text-white py-6 mt-12">
    <div class="container mx-auto px-4 text-center text-gray-400">
      <p>&copy; {{ name }} All rights reserved.</p>
    </div>
  </footer>
</body>
</html>
"""
with open("templates/portfolio.html", "w") as f:
    f.write(portfolio_html_content)


@app.get("/", response_class=HTMLResponse)
async def read_root(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})


@app.post("/generate")
async def generate_site(
    request: Request,
    name: str = Form(...),
    bio: str = Form(...),
    art_style: str = Form(...),
    files: list[UploadFile] = File(...)
):
    os.makedirs("static/images", exist_ok=True)
    artworks_data = []

    for file in files:
        file_path = f"static/images/{file.filename}"
        async with aiofiles.open(file_path, 'wb') as out_file:
            content = await file.read()
            await out_file.write(content)

        image = PIL.Image.open(io.BytesIO(content))

        artwork_prompt = f"""You're an expert creative content writer and SEO specialist for artists, specializing in {art_style} art.
Given the artwork image, generate the following:
- A catchy and professional artwork title
- A 2-3 sentence art description for a portfolio or gallery
- 5 relevant SEO tags
Make it poetic and audience-friendly and reflect the style of {art_style}.
"""
        # Add conditional logic for specific styles
        if "abstract" in art_style.lower():
            artwork_prompt += "\nIf the style is abstract, focus on the emotions, colors, and forms conveyed."
        elif "portraiture" in art_style.lower():
            artwork_prompt += "\nIf the style is portraiture, highlight the subject's expression, personality, and the details of the rendering."
        elif "landscape" in art_style.lower():
             artwork_prompt += "\nIf the style is landscape, describe the atmosphere, light, and the sense of place."


        artwork_content = generate_portfolio_section(artwork_prompt, image=image)

        # Basic parsing of the generated content - refine this based on actual model output format
        title = "Generated Title"
        description = "Generated Description"
        tags = ["tag1", "tag2"] # Example tags

        title_match = re.search(r"(?i)title[:\-\n]*([^\n]+)", artwork_content)
        desc_match = re.search(r"(?i)description[:\-\n]*([\s\S]+?)tags[:\n]", artwork_content)
        tags_match = re.search(r"(?i)tags[:\-\n]*([^\n]+)", artwork_content)

        if title_match:
            title = title_match.group(1).strip()
        if desc_match:
            description = desc_match.group(1).strip()
        if tags_match:
            tags = [tag.strip() for tag in tags_match.group(1).split(",")]

        artworks_data.append({
            "url": file_path,
            "title": title,
            "description": description,
            "tags": tags
        })


    about_text = generate_portfolio_section(f"Write an engaging 'About the Artist' section for an artist named {name}, who specializes in {art_style}. Use an elegant tone.")
    description_text = generate_portfolio_section(f"Write a short art style summary for {art_style}. Focus on what makes this craft meaningful.")

    return templates.TemplateResponse("portfolio.html", {
        "request": request,
        "name": name,
        "bio": bio,
        "about": about_text,
        "style": art_style,
        "description": description_text,
        "artworks": artworks_data
    })

## Summary:

### Data Analysis Key Findings
*   The FastAPI application was successfully structured into `main.py` and `ai_utils.py` for better modularity.
*   The application can handle multiple image uploads, save them locally to the `static/images` directory, and obtain their file paths.
*   The AI utility function `generate_portfolio_section` was enhanced to accept image input using `PIL.Image`.
*   The application utilizes the Gemini Flash 1.5 API to generate titles, descriptions, and tags for individual artworks based on the uploaded images and user-provided information (name, bio, art style).
*   Basic parsing of the AI's output is implemented using regular expressions to extract the title, description, and tags for each artwork.
*   An HTML form (`index.html`) was created to capture user input (name, bio, art style) and image uploads.
*   The FastAPI application serves the `index.html` form at the root URL (`/`) and processes the form submission at the `/generate` endpoint.
*   The generated portfolio website (`portfolio.html`) dynamically displays the user's information and the AI-generated content (title, description, tags) alongside the uploaded images.
*   Tailwind CSS was integrated into the HTML templates to enhance the visual styling and structure of the portfolio website.
*   The AI prompt for generating artwork content was fine-tuned to include the user's specified art style and conditional instructions for specific styles (abstract, portraiture, landscape) to improve content relevance.

### Insights or Next Steps
*   Refine the parsing of the AI's generated content to handle potential variations in output format more robustly, potentially using structured output formats if the model supports them.
*   Implement error handling for file uploads (e.g., file size limits, unsupported file types) and AI generation failures.


**Reasoning**:
Create the index.html file with the necessary form elements and modify main.py to serve this file at the root URL.

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>Generate Your Art Portfolio</title>
  <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
</head>
<body class="bg-white text-gray-800 p-8">
  <div class="max-w-2xl mx-auto">
    <h1 class="text-3xl font-bold mb-6">Generate Your Art Portfolio</h1>

    <form action="/generate" method="post" enctype="multipart/form-data" class="space-y-4">
      <div>
        <label for="name" class="block text-sm font-medium text-gray-700">Artist Name:</label>
        <input type="text" id="name" name="name" required class="mt-1 block w-full rounded-md border-gray-300 shadow-sm focus:border-indigo-500 focus:ring-indigo-500 sm:text-sm">
      </div>

      <div>
        <label for="bio" class="block text-sm font-medium text-gray-700">Short Bio:</label>
        <textarea id="bio" name="bio" rows="3" required class="mt-1 block w-full rounded-md border-gray-300 shadow-sm focus:border-indigo-500 focus:ring-indigo-500 sm:text-sm"></textarea>
      </div>

      <div>
        <label for="art_style" class="block text-sm font-medium text-gray-700">Art Style:</label>
        <input type="text" id="art_style" name="art_style" required class="mt-1 block w-full rounded-md border-gray-300 shadow-sm focus:border-indigo-500 focus:ring-indigo-500 sm:text-sm">
      </div>

      <div>
        <label for="files" class="block text-sm font-medium text-gray-700">Upload Images:</label>
        <input type="file" id="files" name="files" multiple accept="image/*" required class="mt-1 block w-full text-sm text-gray-500 file:mr-4 file:py-2 file:px-4 file:rounded-full file:border-0 file:text-sm file:font-semibold file:bg-blue-50 file:text-blue-700 hover:file:bg-blue-100">
      </div>

      <div>
        <button type="submit" class="w-full flex justify-center py-2 px-4 border border-transparent rounded-md shadow-sm text-sm font-medium text-white bg-indigo-600 hover:bg-indigo-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-indigo-500">
          Generate Portfolio
        </button>
      </div>
    </form>
  </div>
</body>
</html>

Overwriting templates/index.html


## Final Code

Here is the complete code for the AI Art Portfolio Generator.

**`ai_utils.py`**

In [ ]:
%%writefile ai_utils.py
import google.generativeai as genai
import os
from typing import List, Optional
from PIL import Image
import PIL.ExifTags
from google.colab import userdata # Import userdata

# Initialize Gemini Flash 1.5
# Use userdata.get to access the API key from Colab Secrets
genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-flash")

# 🖼️ Helper: Extract basic EXIF description (title, artist, etc.)
def extract_image_metadata(image_path: str) -> str:
    try:
        img = Image.open(image_path)
        exif = {
            PIL.ExifTags.TAGS.get(tag): val
            for tag, val in img._getexif().items()
            if tag in PIL.ExifTags.TAGS
        }
        return exif.get("ImageDescription", "")
    except Exception:
        return ""

# 🧠 Prompt generators
def generate_about_section(name: str, art_style: str, bio: str, language: str = "en") -> str:
    prompt = f"""
    Write a warm, engaging 'About the Artist' section for a handcrafted artist named {name}.
    Their art style is {art_style}. Bio: {bio}.
    Use a narrative tone that feels authentic and personal. Limit to 150-200 words.
    {'Translate the output to Hindi.' if language == 'hi' else ''}
    """
    return model.generate_content(prompt).text.strip()

def generate_style_summary(art_style: str, language: str = "en") -> str:
    prompt = f"""
    Describe the handcrafted art style called '{art_style}' in simple terms.
    Explain its cultural or aesthetic value and what makes it unique.
    Make it understandable for buyers new to handcrafted art.
    Limit to 100-120 words.
    {'Translate the output to Hindi.' if language == 'hi' else ''}
    """
    return model.generate_content(prompt).text.strip()

def generate_image_captions(image_titles: List[str], art_style: str, metadata: List[str], language: str = "en") -> List[str]:
    metadata_text = ", ".join([f"{title} ({desc})" if desc else title for title, desc in zip(image_titles, metadata)])
    prompt = f"""
    Write short, poetic descriptions (2-3 sentences) for the following handcrafted artworks in the {art_style} style:
    {metadata_text}.
    The tone should be emotional and inspiring.
    {'Translate the output to Hindi.' if language == 'hi' else ''}
    """
    response = model.generate_content(prompt)
    return [line.strip("• ").strip() for line in response.text.strip().split("\n") if line.strip()]

# 🔄 Master function
def generate_artist_portfolio(
    name: str,
    bio: str,
    art_style: str,
    image_paths: List[str],
    alt_texts: Optional[List[str]] = None,
    language: str = "en"
) -> dict:
    image_titles = [os.path.basename(path).split(".")[0] for path in image_paths]

    # Use EXIF or fallback alt_texts
    metadata = []
    for i, path in enumerate(image_paths):
        meta = extract_image_metadata(path)
        if not meta and alt_texts:
            meta = alt_texts[i]
        metadata.append(meta)

    return {
        "about": generate_about_section(name, art_style, bio, language),
        "style_summary": generate_style_summary(art_style, language),
        "captions": generate_image_captions(image_titles, art_style, metadata, language)
    }

Overwriting ai_utils.py


**`templates/index.html`**

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>Generate Your Art Portfolio</title>
  <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
</head>
<body class="bg-white text-gray-800 p-8">
  <div class="max-w-2xl mx-auto">
    <h1 class="text-3xl font-bold mb-6">Generate Your Art Portfolio</h1>

    <form action="/generate" method="post" enctype="multipart/form-data" class="space-y-4">
      <div>
        <label for="name" class="block text-sm font-medium text-gray-700">Artist Name:</label>
        <input type="text" id="name" name="name" required class="mt-1 block w-full rounded-md border-gray-300 shadow-sm focus:border-indigo-500 focus:ring-indigo-500 sm:text-sm">
      </div>

      <div>
        <label for="bio" class="block text-sm font-medium text-gray-700">Short Bio:</label>
        <textarea id="bio" name="bio" rows="3" required class="mt-1 block w-full rounded-md border-gray-300 shadow-sm focus:border-indigo-500 focus:ring-indigo-500 sm:text-sm"></textarea>
      </div>

      <div>
        <label for="art_style" class="block text-sm font-medium text-gray-700">Art Style:</label>
        <input type="text" id="art_style" name="art_style" required class="mt-1 block w-full rounded-md border-gray-300 shadow-sm focus:border-indigo-500 focus:ring-indigo-500 sm:text-sm">
      </div>

      <div>
        <label for="files" class="block text-sm font-medium text-gray-700">Upload Images:</label>
        <input type="file" id="files" name="files" multiple accept="image/*" required class="mt-1 block w-full text-sm text-gray-500 file:mr-4 file:py-2 file:px-4 file:rounded-full file:border-0 file:text-sm file:font-semibold file:bg-blue-50 file:text-blue-700 hover:file:bg-blue-100">
      </div>

      <div>
        <button type="submit" class="w-full flex justify-center py-2 px-4 border border-transparent rounded-md shadow-sm text-sm font-medium text-white bg-indigo-600 hover:bg-indigo-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-indigo-500">
          Generate Portfolio
        </button>
      </div>
    </form>
  </div>
</body>
</html>

Overwriting templates/index.html


**`templates/portfolio.html`**

In [ ]:
%%writefile templates/portfolio.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>{{ name }} | Portfolio</title>
  <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
</head>
<body class="bg-gray-100 text-gray-800 font-sans leading-normal tracking-normal">
  <header class="bg-white shadow py-6">
    <div class="container mx-auto px-4 max-w-5xl">
      <h1 class="text-4xl font-bold text-gray-900">{{ name }}</h1>
      <p class="mt-2 italic text-sm text-gray-600">{{ bio }}</p>
    </div>
  </header>

  <main class="container mx-auto px-4 py-8 max-w-5xl">
    <section class="mb-12">
      <h2 class="text-3xl font-semibold text-gray-900 mb-4 border-b-2 border-indigo-500 pb-2">About Me</h2>
      <p class="text-lg text-gray-700 leading-relaxed">{{ about }}</p>
    </section>

    <section class="mb-12">
      <h2 class="text-3xl font-semibold text-gray-900 mb-4 border-b-2 border-indigo-500 pb-2">Art Style: {{ style }}</h2>
      <p class="text-lg text-gray-700 leading-relaxed">{{ description }}</p>
    </section>

    <section>
      <h2 class="text-3xl font-semibold text-gray-900 mb-6 border-b-2 border-indigo-500 pb-2">Gallery</h2>
      <div class="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-8">
        {% for artwork in artworks %}
        <div class="bg-white rounded-xl shadow-lg overflow-hidden transform transition duration-500 hover:scale-105">
          <img src="{{ artwork.url }}" class="w-full h-64 object-cover" alt="Artwork by {{ name }}" />
          <div class="p-6">
            <h3 class="text-xl font-semibold text-gray-900 mb-2">{{ artwork.title }}</h3>
            <p class="text-gray-700 text-sm mb-4">{{ artwork.description }}</p>
            <p class="text-xs text-gray-600">Tags: <span class="font-medium">{{ artwork.tags | join(', ') }}</span></p>
          </div>
        </div>
        {% endfor %}
      </div>
    </section>
  </main>

  <footer class="bg-gray-800 text-white py-6 mt-12">
    <div class="container mx-auto px-4 text-center text-gray-400">
      <p>&copy; {{ name }} All rights reserved.</p>
    </div>
  </footer>
</body>
</html>

Overwriting templates/portfolio.html


**`main.py`**

In [ ]:
%%writefile main.py
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from starlette.templating import Jinja2Templates
from starlette.requests import Request
import os
from ai_utils import generate_portfolio_section # Assuming generate_portfolio_section is sufficient for individual image analysis
import aiofiles
import PIL.Image
import io
import re


app = FastAPI()
os.makedirs("static", exist_ok=True)
os.makedirs("templates", exist_ok=True)
app.mount("/static", StaticFiles(directory="static"), name="static")
templates = Jinja2Templates(directory="templates")

# Create the portfolio.html file (using the enhanced version from the previous step)
portfolio_html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>{{ name }} | Portfolio</title>
  <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
</head>
<body class="bg-gray-100 text-gray-800 font-sans leading-normal tracking-normal">
  <header class="bg-white shadow py-6">
    <div class="container mx-auto px-4 max-w-5xl">
      <h1 class="text-4xl font-bold text-gray-900">{{ name }}</h1>
      <p class="mt-2 italic text-sm text-gray-600">{{ bio }}</p>
    </div>
  </header>

  <main class="container mx-auto px-4 py-8 max-w-5xl">
    <section class="mb-12">
      <h2 class="text-3xl font-semibold text-gray-900 mb-4 border-b-2 border-indigo-500 pb-2">About Me</h2>
      <p class="text-lg text-gray-700 leading-relaxed">{{ about }}</p>
    </section>

    <section class="mb-12">
      <h2 class="text-3xl font-semibold text-gray-900 mb-4 border-b-2 border-indigo-500 pb-2">Art Style: {{ style }}</h2>
      <p class="text-lg text-gray-700 leading-relaxed">{{ description }}</p>
    </section>

    <section>
      <h2 class="text-3xl font-semibold text-gray-900 mb-6 border-b-2 border-indigo-500 pb-2">Gallery</h2>
      <div class="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-8">
        {% for artwork in artworks %}
        <div class="bg-white rounded-xl shadow-lg overflow-hidden transform transition duration-500 hover:scale-105">
          <img src="{{ artwork.url }}" class="w-full h-64 object-cover" alt="Artwork by {{ name }}" />
          <div class="p-6">
            <h3 class="text-xl font-semibold text-gray-900 mb-2">{{ artwork.title }}</h3>
            <p class="text-gray-700 text-sm mb-4">{{ artwork.description }}</p>
            <p class="text-xs text-gray-600">Tags: <span class="font-medium">{{ artwork.tags | join(', ') }}</span></p>
          </div>
        </div>
        {% endfor %}
      </div>
    </section>
  </main>

  <footer class="bg-gray-800 text-white py-6 mt-12">
    <div class="container mx-auto px-4 text-center text-gray-400">
      <p>&copy; {{ name }} All rights reserved.</p>
    </div>
  </footer>
</body>
</html>
"""
with open("templates/portfolio.html", "w") as f:
    f.write(portfolio_html_content)


@app.get("/", response_class=HTMLResponse)
async def read_root(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

# Modified endpoint to handle single image upload
@app.post("/upload-image")
async def upload_image(
    file: UploadFile = File(...)
):
    os.makedirs("static/images", exist_ok=True)
    file_path = f"static/images/{file.filename}"
    async with aiofiles.open(file_path, 'wb') as out_file:
        content = await file.read()
        await out_file.write(content)

    # You might want to return some information about the uploaded image,
    # like its URL, to the frontend.
    return {"filename": file.filename, "url": file_path}


# Modified /generate endpoint to accept image URLs instead of files
@app.post("/generate")
async def generate_site(
    request: Request,
    name: str = Form(...),
    bio: str = Form(...),
    art_style: str = Form(...),
    image_urls: List[str] = Form(...) # Expect a list of image URLs from the frontend
):
    artworks_data = []

    for url in image_urls:
        # To process the image with AI, you would need to read the image file again
        # from the saved path.
        file_path = url # Assuming the URL is the file path
        try:
            image = PIL.Image.open(file_path)

            artwork_prompt = f"""You're an expert creative content writer and SEO specialist for artists, specializing in {art_style} art.
Given the artwork image, generate the following:
- A catchy and professional artwork title
- A 2-3 sentence art description for a portfolio or gallery
- 5 relevant SEO tags
Make it poetic and audience-friendly and reflect the style of {art_style}.
"""
            # Add conditional logic for specific styles
            if "abstract" in art_style.lower():
                artwork_prompt += "\nIf the style is abstract, focus on the emotions, colors, and forms conveyed."
            elif "portraiture" in art_style.lower():
                artwork_prompt += "\nIf the style is portraiture, highlight the subject's expression, personality, and the details of the rendering."
            elif "landscape" in art_style.lower():
                 artwork_prompt += "\nIf the style is landscape, describe the atmosphere, light, and the sense of place."


            artwork_content = generate_portfolio_section(artwork_prompt, image=image)

            # Basic parsing of the generated content - refine this based on actual model output format
            title = "Generated Title"
            description = "Generated Description"
            tags = ["tag1", "tag2"] # Example tags

            title_match = re.search(r"(?i)title[:\-\n]*([^\n]+)", artwork_content)
            desc_match = re.search(r"(?i)description[:\-\n]*([\s\S]+?)tags[:\n]", artwork_content)
            tags_match = re.search(r"(?i)tags[:\-\n]*([^\n]+)", artwork_content)

            if title_match:
                title = title_match.group(1).strip()
            if desc_match:
                description = desc_match.group(1).strip()
            if tags_match:
                tags = [tag.strip() for tag in tags_match.group(1).split(",")]

            artworks_data.append({
                "url": url,
                "title": title,
                "description": description,
                "tags": tags
            })
        except FileNotFoundError:
             # Handle case where the file was not found (e.g., if not uploaded correctly)
             artworks_data.append({
                "url": url,
                "title": "Error: Image not found",
                "description": "Could not process image.",
                "tags": []
            })


    about_text = generate_portfolio_section(f"Write an engaging 'About the Artist' section for an artist named {name}, who specializes in {art_style}. Use an elegant tone.")
    description_text = generate_portfolio_section(f"Write a short art style summary for {art_style}. Focus on what makes this craft meaningful.")

    return templates.TemplateResponse("portfolio.html", {
        "request": request,
        "name": name,
        "bio": bio,
        "about": about_text,
        "style": art_style,
        "description": description_text,
        "artworks": artworks_data
    })

Overwriting main.py


In [ ]:
import google.generativeai as genai
import os
from typing import List, Optional
from PIL import Image
import PIL.ExifTags
from google.colab import userdata # Import userdata

# Initialize Gemini Flash 1.5
# Use userdata.get to access the API key from Colab Secrets
genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-flash")

# 🖼️ Helper: Extract basic EXIF description (title, artist, etc.)
def extract_image_metadata(image_path: str) -> str:
    try:
        img = Image.open(image_path)
        exif = {
            PIL.ExifTags.TAGS.get(tag): val
            for tag, val in img._getexif().items()
            if tag in PIL.ExifTags.TAGS
        }
        return exif.get("ImageDescription", "")
    except Exception:
        return ""

# 🧠 Prompt generators
def generate_about_section(name: str, art_style: str, bio: str, language: str = "en") -> str:
    prompt = f"""
    Write a warm, engaging 'About the Artist' section for a handcrafted artist named {name}.
    Their art style is {art_style}. Bio: {bio}.
    Use a narrative tone that feels authentic and personal. Limit to 150-200 words.
    {'Translate the output to Hindi.' if language == 'hi' else ''}
    """
    return model.generate_content(prompt).text.strip()

def generate_style_summary(art_style: str, language: str = "en") -> str:
    prompt = f"""
    Describe the handcrafted art style called '{art_style}' in simple terms.
    Explain its cultural or aesthetic value and what makes it unique.
    Make it understandable for buyers new to handcrafted art.
    Limit to 100-120 words.
    {'Translate the output to Hindi.' if language == 'hi' else ''}
    """
    return model.generate_content(prompt).text.strip()

def generate_image_captions(image_titles: List[str], art_style: str, metadata: List[str], language: str = "en") -> List[str]:
    metadata_text = ", ".join([f"{title} ({desc})" if desc else title for title, desc in zip(image_titles, metadata)])
    prompt = f"""
    Write short, poetic descriptions (2-3 sentences) for the following handcrafted artworks in the {art_style} style:
    {metadata_text}.
    The tone should be emotional and inspiring.
    {'Translate the output to Hindi.' if language == 'hi' else ''}
    """
    response = model.generate_content(prompt)
    return [line.strip("• ").strip() for line in response.text.strip().split("\n") if line.strip()]

# 🔄 Master function
def generate_artist_portfolio(
    name: str,
    bio: str,
    art_style: str,
    image_paths: List[str],
    alt_texts: Optional[List[str]] = None,
    language: str = "en"
) -> dict:
    image_titles = [os.path.basename(path).split(".")[0] for path in image_paths]

    # Use EXIF or fallback alt_texts
    metadata = []
    for i, path in enumerate(image_paths):
        meta = extract_image_metadata(path)
        if not meta and alt_texts:
            meta = alt_texts[i]
        metadata.append(meta)

    return {
        "about": generate_about_section(name, art_style, bio, language),
        "style_summary": generate_style_summary(art_style, language),
        "captions": generate_image_captions(image_titles, art_style, metadata, language)
    }

In [ ]:
result = generate_artist_portfolio(
    name="Radhika Verma",
    bio="A self-taught terracotta sculptor from Gujarat.",
    art_style="Terracotta Clay Sculptures",
    image_paths=["/content/image-from-rawpixel-id-426278-jpeg (1).jpg", "uploads/tribal_mask.jpg"],
    alt_texts=["Ganesha idol made of natural red clay", "A tribal mask with bold carvings"],
    language="hi"  # Hindi translation
)


In [ ]:
!pip install google-generativeai pillow gradio --quiet


In [ ]:
import os
import google.generativeai as genai

# 🔐 Paste your Gemini API key here
os.environ["GOOGLE_API_KEY"] = "your_gemini_flash_1_5_api_key"
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel("gemini-1.5-flash")


In [ ]:
from typing import List, Optional
from PIL import Image, ExifTags

def extract_exif_description(image: Image.Image) -> str:
    try:
        exif = {
            ExifTags.TAGS.get(tag): val
            for tag, val in image._getexif().items()
            if tag in ExifTags.TAGS
        }
        return exif.get("ImageDescription", "")
    except:
        return ""

def generate_gemini_response(prompt: str) -> str:
    return model.generate_content(prompt).text.strip()

def generate_portfolio(name: str, bio: str, art_style: str, language: str,
                       images: List[Image.Image], alt_texts: Optional[str]) -> dict:

    # Language helper
    def lang_suffix():
        if language == 'hi': return "Translate the output to Hindi."
        if language == 'bn': return "Translate the output to Bengali."
        if language == 'ta': return "Translate the output to Tamil."
        return ""

    # 1. About Section
    about_prompt = f"""
    Write a warm, engaging 'About the Artist' section for a handcrafted artist named {name}.
    Their art style is {art_style}. Bio: {bio}.
    Use a narrative tone. Limit to 150-200 words.
    {lang_suffix()}
    """
    about = generate_gemini_response(about_prompt)

    # 2. Style Summary
    style_prompt = f"""
    Describe the handcrafted art style '{art_style}' in simple, buyer-friendly terms.
    Keep it within 100-120 words.
    {lang_suffix()}
    """
    style_summary = generate_gemini_response(style_prompt)

    # 3. Captions
    alt_text_list = [x.strip() for x in alt_texts.split(",")] if alt_texts else []
    captions_input = []
    for i, image in enumerate(images):
        title = f"Artwork {i+1}"
        meta = extract_exif_description(image)
        if not meta and i < len(alt_text_list):
            meta = alt_text_list[i]
        captions_input.append(f"{title} ({meta})" if meta else title)

    caption_prompt = f"""
    Generate poetic 2-3 sentence descriptions for these artworks in {art_style} style:
    {', '.join(captions_input)}.
    {lang_suffix()}
    """
    captions_output = generate_gemini_response(caption_prompt)
    captions = [line.strip("• ").strip() for line in captions_output.split("\n") if line.strip()]

    return {
        "About the Artist": about,
        "Art Style Summary": style_summary,
        "Captions": captions
    }


In [ ]:
import gradio as gr
from PIL import Image

def ui_interface(name, bio, art_style, language, images, alt_texts):
    # Open images from file paths
    pil_images = [Image.open(image_path) for image_path in images]
    result = generate_portfolio(name, bio, art_style, language, pil_images, alt_texts)
    captions_html = "<ul>" + "".join(f"<li>{cap}</li>" for cap in result["Captions"]) + "</ul>"
    return result["About the Artist"], result["Art Style Summary"], captions_html

gr.Interface(
    fn=ui_interface,
    inputs=[
        gr.Textbox(label="Artist Name"),
        gr.Textbox(label="Bio", lines=3),
        gr.Textbox(label="Art Style"),
        gr.Dropdown(["en", "hi", "bn", "ta"], label="Language", value="en"),
        gr.File(label="Upload Images", file_types=["image"], file_count="multiple", type="filepath"), # Changed type to 'filepath'
        gr.Textbox(label="Alt Texts (comma-separated, optional)", lines=2)
    ],
    outputs=[
        gr.Textbox(label="About the Artist"),
        gr.Textbox(label="Art Style Summary"),
        gr.HTML(label="Artwork Captions")
    ],
    title="🎨 Acrilc AI Portfolio Generator",
    description="Upload your handcrafted art and let AI write your artist bio, captions, and style intro!",
    theme="default"
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a886b2700439b32a21.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install google-generativeai pillow --quiet


In [ ]:
import os
import google.generativeai as genai

# 🔐 Add your Gemini API key
os.environ["GOOGLE_API_KEY"] = "your_gemini_flash_1_5_api_key"
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-1.5-flash")


In [ ]:
from google.colab import files
uploaded = files.upload()
image_paths = list(uploaded.keys())  # store the filenames
